## ARC 380 / CEE 380 – Introduction to Robotics for Digital Fabrication
## Session 16 Workshop
Princeton University, Spring 2024

Professor: Arash Adel | Assistant-in-Instruction: Daniel Ruan

---

# 1. Open3D

"Open3D is an open-source library that supports rapid development of software that deals with 3D data." [1]

Sources:
- [1] https://www.open3d.org/

In [1]:
# %pip install open3d
# %pip install scipy

In [2]:
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


---

# 2. RANSAC

Create a point cloud that simulates a plane with noisy measurements.

In [3]:
# Parameters
width, height = 50, 50  # Size of the plane
noise_sigma = 0.01  # Noise standard deviation
A, B, C, D = 1.0, 0.0, 1.0, 0.0  # Plane equation: Ax + By + Cz + D = 0

# Generate a mesh grid
x = np.linspace(-0.5, 0.5, width)
y = np.linspace(-0.5, 0.5, height)
xx, yy = np.meshgrid(x, y)
zz = (-A * xx - B * yy - D) / C

# Flatten the grid to N x 3 points (N = width x height)
points = np.vstack((xx.flatten(), yy.flatten(), zz.flatten())).T

# Add Gaussian noise
points_noisy = points + np.random.normal(0, noise_sigma, points.shape)

# Create Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_noisy)
pcd.paint_uniform_color([0, 0, 1])  # Blue

# Visualize the point cloud
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
o3d.visualization.draw_geometries([pcd, coordinate_frame])

Define the RANSAC plane-fitting function.

In [4]:
def fit_plane_RANSAC(points, num_iterations=1000, distance_threshold=0.01):
    """
    Fits a plane to a set of 3D points using the RANSAC algorithm.

    Parameters:
    - points: numpy array of shape (N, 3) representing the 3D points.
    - num_iterations: number of iterations to perform RANSAC (default: 1000).
    - distance_threshold: maximum distance from the plane for a point to be considered an inlier (default: 0.01).

    Returns:
    - best_plane: tuple (normal_vector, d) representing the best-fit plane, where normal_vector is the normal vector of the plane and d is the distance from the origin.
    - inlier_points: numpy array of shape (M, 3) representing the inlier points that lie close to the plane.
    """

    max_inliers = []
    best_plane = None

    for i in range(num_iterations):
        # Step 1: Randomly select 3 points
        sample_indices = np.random.choice(points.shape[0], 3, replace=False)
        sample_points = points[sample_indices, :]

        # Step 2: Compute the plane defined by these three points
        v1 = sample_points[1] - sample_points[0]
        v2 = sample_points[2] - sample_points[0]
        normal_vector = np.cross(v1, v2)
        normal_vector = normal_vector / np.linalg.norm(normal_vector)  # Normalize the vector
        d = -np.dot(normal_vector, sample_points[0]) # Compute d in the plane equation (Projection of the first point on the normal vector)

        # Step 3: Count inliers, i.e., points close to the plane
        distances = np.abs(np.dot(points, normal_vector) + d)
        inliers = np.where(distances < distance_threshold)[0]

        # Step 4: Check if the current model is the best one
        if len(inliers) > len(max_inliers):
            max_inliers = inliers
            best_plane = (normal_vector, d)

    return best_plane, points[max_inliers]

Run the RANSAC function on our point cloud and visualize the best-fit plane. Edit `noise_sigma` and the plane equation in the first cell to test the capabilities of RANSAC. (You may also need to edit the `distance_threshold`.)

Also note how quickly the code runs even though we are iterating 1000 times.

In [5]:
# Time the plane-fitting RANSAC function
import time
start_time = time.time()
plane, inliers = fit_plane_RANSAC(np.asarray(pcd.points), num_iterations=1000, distance_threshold=0.01)
print(f'RANSAC execution time: {time.time() - start_time: .5f} seconds.')

print(f'{len(inliers)} inliers out of {len(pcd.points)} points.')
print(f'Plane normal: {plane[0]}')
print(f'Plane d: {plane[1]}')

# Create a mesh for the plane
from helper import create_plane_mesh
plane_mesh = create_plane_mesh(plane, size=1.5)  # Adjust 'size' as needed
plane_mesh.paint_uniform_color([0.8, 0.8, 0.8])

# Visualizing the plane with the point cloud
o3d.visualization.draw_geometries([pcd, plane_mesh, coordinate_frame])

RANSAC execution time:  0.13097 seconds.
1707 inliers out of 2500 points.
Plane normal: [-0.70912758 -0.00491794 -0.70506304]
Plane d: -0.0005978101073474203


Our implementation of RANSAC is very basic and not the most efficient, but it illustrates the primary steps. There are functions for RANSAC built into Open3D that can run RANSAC much faster, such as `segment_plane`. So much faster that we need to use a dedicated library to measure it.

In [6]:
# Define a wrapper that runs the Open3D RANSAC function
def wrapper():
    pcd.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)

# Run the function 1000 times and calculate the average time taken using timeit
import timeit
n = 1000
total_time_taken = timeit.timeit('wrapper()', globals=globals(), number=n)
avg_time_taken = total_time_taken / n

print(f'Open3D RANSAC average execution time: {avg_time_taken} seconds.')

Open3D RANSAC average execution time: 0.00021945510000000114 seconds.


---

# 3. ICP

For our ICP example, we will be using the Stanford bunny model, which is widely used as a test model for 3D computer graphics.

Source: http://graphics.stanford.edu/data/3Dscanrep/

In [47]:
# Load the Stanford bunny point cloud
bunny_model = o3d.io.read_point_cloud("bunny.ply")

# Visualize the bunny and the scene
o3d.visualization.draw_geometries([bunny_model])

We will need some code from helper.py for generating the example.

In [48]:
from helper import random_transformation

To simulate scanning, we will make a copy of the bunny (reimport it again) and randomly transform it in the world, which forms the scanned scene.

What we are trying to do with ICP is determine what this random transformation is by matching the bunny model to the bunny in the scene.

In [49]:
# Make a copy of the bunny
bunny_scene = o3d.io.read_point_cloud("bunny.ply")

# Create random 3D rotation matrix and apply it to the scene
T = random_transformation(scale=1)
bunny_scene.transform(T)

# Visualize the bunny in the scene
# The model is in blue, and the scene is in red
bunny_model.paint_uniform_color([0, 0, 1])  # Blue
bunny_scene.paint_uniform_color([1, 0, 0])  # Red
o3d.visualization.draw_geometries([bunny_model, bunny_scene])

[[ 0.72167535 -0.21278266  0.65871711  0.72370259]
 [ 0.49047354  0.82868347 -0.26966537  0.31948244]
 [-0.48848786  0.51769416  0.7024047  -0.61112122]
 [ 0.          0.          0.          1.        ]]


Similar to RANSAC, more efficient versions of ICP have been implemented in Open3D.

In [54]:
# Run the Open3D ICP algorithm
icp_result = o3d.pipelines.registration.registration_icp(bunny_model, bunny_scene, 1.0)

print(icp_result.transformation)

Open3D ICP execution time:  0.44415 seconds.
[[ 0.73349037 -0.20733791  0.64730431  0.72238997]
 [ 0.45072586  0.86120549 -0.23488571  0.31578128]
 [-0.50876131  0.4640432   0.72513849 -0.60636178]
 [ 0.          0.          0.          1.        ]]


We can transform the scene model and visualize the result.

In [55]:
from copy import deepcopy
# Make a copy of the bunny model and apply the ICP transformation
bunny_model_icp = deepcopy(bunny_model)
bunny_model_icp.transform(icp_result.transformation)

PointCloud with 35947 points.

In [57]:
# Visualize the bunny model and the scene after ICP
o3d.visualization.draw_geometries([bunny_model_icp, bunny_scene])